<a href="https://colab.research.google.com/github/Raj9984/Legal_Doc_Summarization-pegasus-billsum-/blob/main/Legal_Doc_Summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.0/58.0 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.9/138.9 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━

In [ ]:
from transformers import pipeline, AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_metric, DatasetDict
import pandas as pd
import torch
from tqdm import tqdm

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "google/pegasus-cnn_dailymail"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [ ]:
def generate_batch_sized_chunks(list_of_elements, batch_size):
    """split the dataset into smaller batches that we can process simultaneously
    Yield successive batch-sized chunks from list_of_elements."""
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]

In [ ]:
def calculate_metric_on_test_ds(dataset, metric, model, tokenizer,
                               batch_size=16, device=device,
                               column_text="article",
                               column_summary="highlights"):
    article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
    target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):

        inputs = tokenizer(article_batch, max_length=1024,  truncation=True,
                        padding="max_length", return_tensors="pt")

        summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                         attention_mask=inputs["attention_mask"].to(device),
                         length_penalty=0.8, num_beams=8, max_length=128)
        ''' parameter for length penalty ensures that the model does not generate sequences that are too long. '''

        # Finally, we decode the generated texts,
        # replace the  token, and add the decoded texts with the references to the metric.
        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                clean_up_tokenization_spaces=True)
               for s in summaries]

        decoded_summaries = [d.replace("", " ") for d in decoded_summaries]


        metric.add_batch(predictions=decoded_summaries, references=target_batch)

    #  Finally compute and return the ROUGE scores.
    score = metric.compute()
    return score

In [ ]:
#dataset used => billsum
#model used => pegasus-cnn_dailymail
dataset = load_dataset("FiscalNote/billsum")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/18949 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3269 [00:00<?, ? examples/s]

Generating ca_test split:   0%|          | 0/1237 [00:00<?, ? examples/s]

In [ ]:

# Split the dataset into train, validation, and test sets
train_test_split = dataset['train'].train_test_split(test_size=0.1)  # 10% for test set

train_val_split = train_test_split['train'].train_test_split(test_size=0.1)  # 10% of remaining for validation set

# Combine splits into a DatasetDict
dataset_split = DatasetDict({
    'train': train_val_split['train'],
    'validation': train_val_split['test'],
    'test': train_test_split['test']
})

# Print the structure of the dataset
print(dataset_split)

DatasetDict({
    train: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 15348
    })
    validation: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 1706
    })
    test: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 1895
    })
})


In [ ]:
rouge_metric = load_metric('rouge')

score = calculate_metric_on_test_ds(dataset_split['test'], rouge_metric, model, tokenizer, column_text = 'text', column_summary='summary', batch_size=8)


100%|██████████| 237/237 [48:16<00:00, 12.22s/it]


In [ ]:
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
rouge_dict = dict((rn, score[rn].mid.fmeasure ) for rn in rouge_names )

pd.DataFrame(rouge_dict, index = ['pegasus'])

,rouge1,rouge2,rougeL,rougeLsum
pegasus,0.023258,0.000489,0.021593,0.022065


In [ ]:
def convert_examples_to_features(example_batch):
    input_encodings = tokenizer(example_batch['text'] , max_length = 1024, truncation = True )

    with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(example_batch['summary'], max_length = 128, truncation = True )

    return {
        'input_ids' : input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids']
    }

dataset_billsum = dataset_split.map(convert_examples_to_features, batched = True)

Map:   0%|          | 0/15348 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4016: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/1706 [00:00<?, ? examples/s]

Map:   0%|          | 0/1895 [00:00<?, ? examples/s]

In [ ]:
dataset_billsum

DatasetDict({
    train: Dataset({
        features: ['text', 'summary', 'title', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 15348
    })
    validation: Dataset({
        features: ['text', 'summary', 'title', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1706
    })
    test: Dataset({
        features: ['text', 'summary', 'title', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1895
    })
})

In [ ]:
from transformers import DataCollatorForSeq2Seq

seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from transformers import TrainingArguments, Trainer

trainer_args = TrainingArguments(
    output_dir='pegasus-billsum', num_train_epochs=1, warmup_steps=500,
    per_device_train_batch_size=1, per_device_eval_batch_size=1,
    weight_decay=0.01, logging_steps=10,
    evaluation_strategy='steps', eval_steps=500, save_steps=100,
    gradient_accumulation_steps=16
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Trainer(model=model, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_billsum["train"],
                  eval_dataset=dataset_billsum["validation"])

In [ ]:
trainer.train()

Step,Training Loss,Validation Loss
500,1.549100,1.398807


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8, 'forced_eos_token_id': 1}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8, 'forced_eos_token_id': 1}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/tran

TrainOutput(global_step=959, training_loss=1.7783532982946562, metrics={'train_runtime': 12216.4492, 'train_samples_per_second': 1.256, 'train_steps_per_second': 0.079, 'total_flos': 4.374734057562931e+16, 'train_loss': 1.7783532982946562, 'epoch': 0.9997393797237425})

In [ ]:
score = calculate_metric_on_test_ds(
    dataset_billsum['test'], rouge_metric, trainer.model, tokenizer, batch_size = 2, column_text = 'text', column_summary= 'summary'
)

rouge_dict = dict((rn, score[rn].mid.fmeasure ) for rn in rouge_names )

pd.DataFrame(rouge_dict, index = [f'pegasus'] )

In [ ]:
## Save model
model.save_pretrained("pegasus-billsum-model")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8, 'forced_eos_token_id': 1}


In [ ]:
## Save tokenizer
tokenizer.save_pretrained("tokenizer")

('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/spiece.model',
 'tokenizer/added_tokens.json',
 'tokenizer/tokenizer.json')

In [ ]:
import shutil

shutil.copytree("pegasus-billsum-model", "/content/drive/My Drive/pegasus-billsum-model")
shutil.copytree("tokenizer", "/content/drive/My Drive/tokenizer")

'/content/drive/My Drive/tokenizer'

## TEST

In [ ]:
%cd /content/drive/MyDrive/Legal_Summarization_Model

/content/drive/MyDrive/Legal_Summarization_Model


In [ ]:
pwd

'/content/drive/MyDrive/Legal_Summarization_Model'

In [ ]:
ls

pegasus-billsum-model/  tokenizer/


In [ ]:
dataset = load_dataset("FiscalNote/billsum")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("tokenizer")

In [ ]:
gen_kwargs = {"length_penalty": 0.8, "num_beams":8, "max_length": 128}

pipe = pipeline("summarization", model="pegasus-billsum-model",tokenizer=tokenizer)

In [ ]:
#finding a short text to check the performance
len_tokens = [len(tokenizer.encode(s)) for s in dataset['test']['text']]

In [ ]:
l = len(len_tokens)
indexes = []
for i in range(l):
  if len_tokens[i]<1024:
    indexes.append(i)
print(indexes)

[12, 20, 25, 29, 31, 32, 40, 48, 56, 61, 63, 77, 83, 101, 106, 122, 127, 128, 134, 147, 160, 170, 179, 180, 182, 210, 217, 224, 225, 242, 249, 252, 262, 286, 291, 320, 328, 333, 341, 343, 350, 354, 360, 362, 364, 377, 380, 382, 386, 393, 400, 401, 419, 427, 429, 431, 449, 463, 469, 482, 488, 491, 495, 496, 499, 510, 526, 532, 536, 541, 542, 548, 552, 553, 554, 560, 563, 569, 572, 578, 589, 596, 597, 615, 616, 622, 629, 631, 633, 636, 644, 649, 652, 654, 664, 669, 681, 684, 685, 696, 697, 708, 721, 730, 732, 734, 737, 739, 740, 741, 751, 753, 757, 759, 763, 766, 775, 780, 791, 797, 800, 804, 813, 823, 834, 842, 845, 852, 865, 887, 893, 909, 910, 912, 919, 923, 932, 934, 943, 947, 951, 961, 963, 969, 983, 986, 988, 991, 996, 997, 1001, 1002, 1009, 1010, 1030, 1034, 1035, 1056, 1057, 1061, 1068, 1089, 1096, 1102, 1103, 1107, 1113, 1125, 1127, 1143, 1145, 1148, 1159, 1172, 1186, 1188, 1192, 1202, 1203, 1211, 1239, 1249, 1253, 1284, 1287, 1303, 1316, 1317, 1319, 1320, 1324, 1325, 1337, 1341

In [ ]:
print(l)

3269


In [ ]:
sample_text = dataset['test'][12]['text']
reference = dataset['test'][12]['title']
print("text:")
print(sample_text)


print("\nReference Summary:")
print(reference)


print("\nModel Summary:")
print(pipe(sample_text, **gen_kwargs)[0]["summary_text"])

text:
SECTION 1. RECOGNITION OF ACCREDITING AGENCIES AND ASSOCIATIONS.

    (a) Due Process.--Paragraph (6) of section 496(a) of the Higher 
Education Act of 1965 (20 U.S.C. 1099b(a)) is amended to read as 
follows:
            ``(6) such agency or association shall apply procedures 
        throughout the accrediting process, including evaluation, 
        interim sanction, and withdrawal or termination of 
        accreditation proceedings, that comply with due process, 
        including the right to--
                    ``(A) adequate specification of the agency's 
                accrediting requirements;
                    ``(B) written notice of deficiencies at the 
                institution of higher education or program being 
                examined and a reasonable time period, expressly set 
                forth in the notice, to correct the deficiencies;
                    ``(C) advance notice of and an opportunity for a 
                hearing by any such institut